<a href="https://colab.research.google.com/github/hariszaf/metabolic_toy_model/blob/main/Antony2025/introductionToCOBRApy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Introduction to COBRApy**

## Prepare your environment

> **NOTE**
> 
> This section is necessary only when you are running this notebook on GoogleCollab! 
> If you are running locally, then you should already have a running `conda` environment as described [here](./preparingYourEnvironment.ipynb).

First, in case you are working on Google Colab, **get the workshop's GitHub repo**, so we can easily access the files that we will need.  

In [ ]:
!git clone https://github.com/hariszaf/metabolic_toy_model.git

Again, only for the case you are running on Google Collab, make the `Antony2025` folder of the repo your working directory. 

In [ ]:
import os
os.chdir("/content/metabolic_toy_model/Antony2025")

Before starting our workshop, let us first make sure everyone gets a Gurobi license! 

> **NOTE**
>
> You will do this once, and then you will activate the same license, using the same credentials across all the different notebooks. 


* If you already have a Gurobi account, click [here](https://portal.gurobi.com/iam/login/) and log-in

* If you do not have an account, you need first to [register](https://portal.gurobi.com/iam/register/). 

Please do so, and let us know when you are logged-in.

Once you are logged-in, you need to make sure you get a Web License Service (WLS) academic license.

To do so, first click on the **Licenses** button on the left and then on the **Request**.

![request](./files/figs/gurobi/request.png)

From the licenses that pop-up, select the WLS option

![request wls](./files/figs/gurobi/request_wls.png)

Once you are authorized with that, you click on the **Licenses** button again and you should now have at least one license there. 
If you already have a WLS, Gurobi would not let you get a second one. However, you can get several Named-User Academic licenses. 

![licenses overview](./files/figs/gurobi/licenses.png)



Finally, by clicking on the open on the Web License Manager button, you will see an overview of your license like this:

![download](./files/figs/gurobi/download.png)

You now have to Download the license, give any name and description of your choice and click download. 
This will bring locally on your computer a `gurobi.lic` file. 

Open it with any text editor of your choice and this should look like this:

```
(base) u0156635@gbw-l-l0074:gurobi$ cat gurobi_wlm.lic 
# Gurobi WLS license file
# Your credentials are private and should not be shared or copied to public repo
sitories.
# Visit https://license.gurobi.com/manager/doc/overview for more information.
WLSACCESSID=d5419c87-0d36-4a93-9385-773f5483b3c1
WLSSECRET=afa5d95f-ad0b-4a38-9550-a8913aacb7c0
LICENSEID=964844
```

Please get those values from your own `gurobi.lic` and complete the following variables:

> **NOTE**
>
> The following chunks are the ones you need to make sure you run across the different notebooks, so you always have a working gurobi. 

In [ ]:
WLSACCESSID = ""
WLSSECRET   = ""
LICENSEID   = ""

In [ ]:
# @title
import os
def create_gurobi_license(WLSACCESSID, WLSSECRET, LICENSEID):
    license_content = (
        "# Gurobi WLS license file\n"
        "# Your credentials are private and should not be shared or copied to public repositories.\n"
        "# Visit https://license.gurobi.com/manager/doc/overview for more information.\n"
        f"WLSACCESSID={WLSACCESSID}\n"
        f"WLSSECRET={WLSSECRET}\n"
        f"LICENSEID={LICENSEID}"
    )
    with open("/content/licenses/gurobi.lic", "w") as f:
        f.write(license_content)
    print("License file created at /content/licenses/gurobi.lic")

# Create directory for the license
os.makedirs("/content/licenses", exist_ok=True)

# Generate the license file
create_gurobi_license(WLSACCESSID, WLSSECRET, LICENSEID)

#add to path
os.environ['GRB_LICENSE_FILE'] = '/content/licenses/gurobi.lic'

In [ ]:
# @title
!pip install gurobipy
!pip install cobra

In [ ]:
# @title
import gurobipy
from gurobipy import Model
model = Model("test")
print("Gurobi is working!", "\U0001F600")

#install COBRApy
import cobra
from cobra.io import load_model
model = load_model("textbook")
solution = model.optimize()
print(f"flux balance analysis solution is {solution.objective_value}")
print("COBRApy is working", "\U0001F600")

## Our three gut species example case

![three species](./files/figs/multistabilityPaper.png)

We will first use three toy models for three human gut species that occupy important ecological niches in the gut and are known to metabolically interact with one another as in the following diagram:

These ecological niches are based on the metabolism of the following three species:

![test](../files/figs/primeferm_but_acet.png)

## Building a metabolic toy model 

### The openCOBRA project

First, load all the necessary libraries for this section. 

In [5]:
import pandas as pd

We will now consider a set of core-metabolism related reactions to build a toy-model, meaning a model that is not representing the actual metabolism of a living species,
but it will be an easy way for us to "jam" a little :sunglasses: 

After this section, you will (have to) be able to:

* parse the metabolites of a `cobra` model 
* parse the reactions of a `cobra` model 
* view and edit the medium of a `cobra` model 
* reach the objective function of the model and get its optimal solution -- we will discuss further this part and its background on [tomorrow's afternoon session](./computationalMethods.ipynb)

But before anything further, **what is `cobra`**? :thinking: \U+1F914

[openCOBRA](https://opencobra.github.io) is an open-source, community-developed code base for COnstraint-Based Reconstruction and Analysis.

Its name is also indicative of our workshop's program: 

* **Reconstruction:** 
  
  how can we represent a species' metabolism in terms of a programming object? How can we link a reaction to its metabolites and to the genes that code for their corresponding enzymes?  
  Such questions and more will be covered in this session. Its application on the genome-scale will be shown in the [exact following session](./reconstructingDraftGSMMs.ipynb).

* **Cosntraint-based Analysis**:

  The definition of constraints and the various types of analyses for metabolic models will be discussed during [tomorrow's afternoon session](./computationalMethods.ipynb). 
  During this last session, we will explore how COBRA methods are actually applied throughout the workshop sessions.


The openCOBRA project supports COBRA methods across several programming languages. For this workshop, we will be using COBRApy, although it's important to note that the project began with MATLAB, and many analyses and routines are still only available in the COBRA Toolbox. However, the Python interface of COBRA offers a more accessible approach for capturing the complexity of integrated biological networks and provides an integration framework for multiomics data in systems biology.

For more on COBRApy you may study its corresponding [publication](https://bmcsystbiol.biomedcentral.com/articles/10.1186/1752-0509-7-74).

What you will need for sure though for this workshop and for your future metabolic modeling adventures in Python is cobra's [documentation page](https://bmcsystbiol.biomedcentral.com/articles/10.1186/1752-0509-7-74).

### Building a metabolic toy-model

In the [`BT_metabolicReactions.txt`](files/BT_metabolicReactions.txt) we have brought together a set of reactions to highlight *Bacteroides thetaiotaomicron*'s (BT) nature as a primary fermenter.

In [28]:
genome_reactions = pd.read_csv("files/BT_metabolicReactions.txt", sep="\t") 
genome_reactions.head()

,ID (BIGG),ID (SEED),name,Reaction,exchange,SEED link,Notes
0,GLCt2r,rxn05573,D-glucose transport in via proton symport,D-Glucose[1] + H+[1] <=> D-Glucose + H+,D-Glucose[1],https://modelseed.org/biochem/reactions/rxn05573,NaN
1,HEX1,rxn00216,ATP:D-glucose 6-phosphotransferase,ATP + D-Glucose => ADP + H+ + D-glucose-6-phos...,NaN,https://modelseed.org/biochem/reactions/rxn00216,NaN
2,PGI,rxn00558,D-glucose-6-phosphate aldose-ketose-isomerase,D-glucose-6-phosphate <=> D-fructose-6-phosphate,NaN,https://modelseed.org/biochem/reactions/rxn00558,NaN
3,PFK,rxn00545,ATP:D-fructose-6-phosphate 1-phosphotransferase,ATP + D-fructose-6-phosphate => ADP + H+ + D-f...,NaN,https://modelseed.org/biochem/reactions/rxn00545,NaN
4,FBA,rxn00786,"D-fructose-1,6-bisphosphate D-glyceraldehyde-3...","D-fructose-1,6-bisphosphate <=> Glycerone-phos...",NaN,https://modelseed.org/biochem/reactions/rxn00786,NaN


As shown above, this file has seven columns from which 
* **BIGG id**

    [BiGG Models](http://bigg.ucsd.edu) is a knowledgebase of genome-scale metabolic network reconstructions that integrates metabolic models building a set of standardized identifiers called BiGG IDs.



* **SEED id**

    [ModelSEED](https://modelseed.org) is a resource for the reconstruction, exploration, comparison, and analysis of metabolic models.
    ModelSEED brings its own namespace for metabolites (`cpd_` prefix) and reactions (`rxn_` prefix). 
    You may [browse on ModelSEED](https://modelseed.org/biochem/reactions) even without being a user, yet by making an account you are able to reconstruct draft models on the fly. 
    Moreover, ModelSEED is integrated with [KBase](https://www.kbase.us),
    a community-driven research platform for systems biology that provides open science tools, data, and computing resources. 

* **Reaction**

    In this column, we can see three important things:
    - products and the reactants participating the reaction
    - the stoichiometry 
    - the **reversibility** of the reaction

The rest of the columns are much easier to get now: under the **name** column, there's the name of the reaction according to ModelSEED ontology, the **reaction** column provides the stoichiometry of it and its *reversibility*, while the **exchange** column shows which metabolite is being exchange between the cell itself (cytosol) and its environment. 
Last, **SEED link** points to the ModelSEED record of the reaction, and **Notes** gives some extra information on what or why we have added a reaction on this set.


In total, this file contains 23 reactions:

In [27]:
genome_reactions.shape

(23, 7)

which will be the basis for the toy model. 
The reconstruction process for this toy-model is out-of-scope of this workshop, but in case you are interested you may have a look at the corresponding [build_bt_model.py](../scripts/build_bt_model.py).

This script's output is a [Systems Biology Markup Language (SBML)](https://sbml.org) file. 
As mention in SBML's page, SBML is a **software data format for describing models** in biology. 
You can think of it as a structure way to denote certain entities like metabolites, reactions, genes and their relationships.

> **Hint!** 
> SBML is **not** tool-dependent or analysis-specific; rather, it serves as a standardized interface for building models that can be used across different system platforms and operating systems. Software and tools for metabolic model analysis typically rely on SBML as the primary format for receiving input, i.e., the metabolic model.

Let us now have a look on the SBML toy-model we built for BT!

### Model's basic parts 

Let us have a look on its content.

In [2]:
%%bash
head -20 ../files/models/sugar_fermenter_toy_model.xml

<?xml version="1.0" encoding="UTF-8"?>
<sbml xmlns="http://www.sbml.org/sbml/level3/version1/core" xmlns:fbc="http://www.sbml.org/sbml/level3/version1/fbc/version2" sboTerm="SBO:0000624" level="3" version="1" fbc:required="false">
  <model metaid="meta_sugar_fermenter" id="sugar_fermenter" fbc:strict="true">
    <listOfUnitDefinitions>
      <unitDefinition id="mmol_per_gDW_per_hr">
        <listOfUnits>
          <unit kind="mole" exponent="1" scale="-3" multiplier="1"/>
          <unit kind="gram" exponent="-1" scale="0" multiplier="1"/>
          <unit kind="second" exponent="-1" scale="0" multiplier="3600"/>
        </listOfUnits>
      </unitDefinition>
    </listOfUnitDefinitions>
    <listOfCompartments>
      <compartment id="e" constant="true"/>
      <compartment id="c" constant="true"/>
    </listOfCompartments>
    <listOfSpecies>
      <species id="M_cpd00027_e" name="D-Glucose" compartment="e" hasOnlySubstanceUnits="false" boundaryCondition="false" constant="false" fbc:ch

Most likely, it does not look that helpful, especially if you think that a genome-scale model would come up with thousands of metabolites and reactions!

And this is the first instance where `cobra` comes to our rescue!

We will use `cobra` to *load* the model in an easily accessible way.

In [52]:
bt_model = cobra.io.read_sbml_model("../files/models/sugar_fermenter_toy_model.xml")
bt_model

Name,sugar_fermenter
Memory address,7b9930918c90
Number of metabolites,37
Number of reactions,33
Number of genes,0
Number of groups,0
Objective expression,1.0*biomass - 1.0*biomass_reverse_01e59
Compartments,"e, c"


This is our model's summary! Our toy model has a name `sugar_fermenter` and consists of 37 metabolites, 33 reactions with one among those being its **objective function** (we will discuss later on what that means) and two compartments. 

So, let's check on those entities one-by-one to have a clear overview of the model! 

### Metabolites in `cobra`

A list that contains an object for each metabolite in the model.

There are different functions to retrieve the desired metabolite object from the model, according to some query criteria:

`model.metabolites.get_by_id` , `model.metabolites.get_by_any`, `model.metabolites.has_id`

The most important attributes of a metabolite object are:

- id - allows one to access the model object.

In [ ]:
water = bt_model.metabolites.cpd00001_c
water

if the database used contains ids that are not suitable as python variable names (such as the BIGG ids), the metabolite can be accessed with the `get_by_id` function of the `model.metabolites` object.


In [ ]:
water = bt_model.metabolites.get_by_id('cpd00001_c') #same result as above
water

Metabolite identifier,cpd00001_c
Name,H2O
Memory address,0x7b9930a99290
Formula,H2O
Compartment,c
In 3 reaction(s),"rxn00459, rxn00799, h2oSink"


Some attributes of the `metabolite` object

In [36]:
#name
print('name: ', water.name)

#formula
print('formula: ', water.formula)

#elements
print('elements: ', water.elements)

#charge
print('charge: ', water.charge)

#compartment (The metabolite ids commonly have a '_c' or '_e' as a suffix, to indicate their compartments)
print('compartment: ', water.compartment)

#reactions (reaction objects where the metabolite is either a reactant or a product)
print('reactions: ', water.reactions)

name:  H2O
formula:  H2O
elements:  {'H': 2, 'O': 1}
charge:  0
compartment:  c
reactions:  frozenset({<Reaction rxn00459 at 0x7b9930af6a10>, <Reaction rxn00799 at 0x7b9930b8ecd0>, <Reaction h2oSink at 0x7b99bd3699d0>})


### Reactions in `cobra`

Like metabolites, reactions are an attribute of the `cobra` model object. 

Consists of a list containing a reaction object for each of the model's reactions.

Let us first see how a reaction looks like:

In [39]:
bt_model.reactions[0]

Reaction identifier,rxn05573
Name,D-glucose transport in via proton symport
Memory address,0x7b993333d450
Stoichiometry,cpd00027_e + cpd00067_e <=> cpd00027_c + cpd00067_c D-Glucose + H+ <=> D-Glucose + H+
GPR,
Lower bound,-1000.0
Upper bound,1000.0


Again, like in the metabolites case, each reaction has a unique id and a name. 

Yet, the most important attributes of a reaction in a model is its stoichiometry, where we also see its **reversibility**, and its **bounds**,

We call bounds the lowest and the higher value the flux of the reaction can get.

In this case, where we have the reactions' flux ranging from `-1000` to `1000` we have the case of a free flux that it may get any possible flux. 

> **TASK** :question:
>
> Could you think what the bounds of an irreversible reaction would look like?

In [ ]:
#number of reactions in the model
print(len(bt_model.reactions))
for r in bt_model.reactions:
    print(r.id, r.build_reaction_string())  # use_metabolite_names=1

Similar to the metabolite objects, reaction objects may be accessed directly by their ids or with the special functions:


`model.reactions.get_by_id` , `model.reactions.get_by_any`, `model.reactions.has_id`


In [ ]:
acetate_ptransferase = bt_model.reactions.rxn00225
print(acetate_ptransferase)
acetate_ptransferase = bt_model.reactions.get_by_id('rxn00225')
print(acetate_ptransferase)

rxn00225: cpd00002_c + cpd00029_e <-- cpd00008_c + cpd00196_c
rxn00225: cpd00002_c + cpd00029_e <-- cpd00008_c + cpd00196_c


Relevant attributes of the reaction object:

In [ ]:
#name
print('name:', acetate_ptransferase.name, '\n')

#compartments (while metabolites are present on a single compartments, reactions may occur across compartments)
print('compartments: ', acetate_ptransferase.compartments, '\n')

#metabolites
print('metabolites: ', acetate_ptransferase.metabolites)
print([i.name for i in acetate_ptransferase.metabolites], '\n')

#reactants
print('reactants: ', acetate_ptransferase.reactants)
print([i.name for i in acetate_ptransferase.reactants], '\n')

#products
print('products: ', acetate_ptransferase.products)
print([i.name for i in acetate_ptransferase.products], '\n')

#reaction
print('reaction: ', acetate_ptransferase.reaction, '\n')

#other ways to see the reaction
print(acetate_ptransferase.build_reaction_string())

print(acetate_ptransferase.build_reaction_string(use_metabolite_names=1))

name: ATP:acetate phosphotransferase 

compartments:  {'e', 'c'} 

metabolites:  {<Metabolite cpd00002_c at 0x7ec21ac2d750>: -1.0, <Metabolite cpd00029_e at 0x7ec21acc7f10>: -1.0, <Metabolite cpd00008_c at 0x7ec21ac2d650>: 1.0, <Metabolite cpd00196_c at 0x7ec21ac4ddd0>: 1.0}
['ATP', 'Acetate', 'ADP', 'Acetylphosphate'] 

reactants:  [<Metabolite cpd00002_c at 0x7ec21ac2d750>, <Metabolite cpd00029_e at 0x7ec21acc7f10>]
['ATP', 'Acetate'] 

products:  [<Metabolite cpd00008_c at 0x7ec21ac2d650>, <Metabolite cpd00196_c at 0x7ec21ac4ddd0>]
['ADP', 'Acetylphosphate'] 

reaction:  cpd00002_c + cpd00029_e <-- cpd00008_c + cpd00196_c 

cpd00002_c + cpd00029_e <-- cpd00008_c + cpd00196_c
ATP + Acetate <-- ADP + Acetylphosphate


In [ ]:
#see the details of the metabolic reactions containing the metabolite
for i in water.reactions:
  print(i.id, '\t', i.build_reaction_string(use_metabolite_names=True), '\n')

Notice that the `reaction.metabolites` object returns a python dictionary where the keys are the metabolite objects that participate in the reactions and the values are their stoichiometries. Reactants and products have, respectively, negative and positive values.

Since this is a constraint-based model, reactions also have lower and upper bounds, which constraint the fluxes. Through these bounds we may cotrol the direction of the reaction.

In [ ]:
print('lower bound: ', acetate_ptransferase.lower_bound, '\n')

print('upper bound: ', acetate_ptransferase.upper_bound, '\n')

# Set the lower bound to zero
acetate_ptransferase.lower_bound = 0
print('updated lower bound: ', acetate_ptransferase.lower_bound, '\n')

print('Default reaction: ', acetate_ptransferase.build_reaction_string(use_metabolite_names=1), '\n')

print("Is my reaction reversible?", acetate_ptransferase.reversibility)

# Still, we can make it if we wish to
print('irreversible reaction: ', acetate_ptransferase.build_reaction_string(use_metabolite_names=1))

lower bound:  0 

upper bound:  0.0 

updated lower bound:  0 

Default reaction:  ATP + Acetate --> ADP + Acetylphosphate 

Is my reaction reversible? False
irreversible reaction:  ATP + Acetate --> ADP + Acetylphosphate


### Compartments

We have already mentioned this concept, yet we have not defined it so far.

In a metabolic model, we need to know **where** a compound is available and **where** a reaction is taking place.

To this end, a model may have several **compartments** to simulate real-world conditions. 
In Bacteria, things are quite straight-forward in this aspect and most of the times we only consider two compartments:
a compound may be available within the cell, in the cytosol (`c`) or in its environment (`e`). 
Of course, you may see other compartments as well, e.g. the periplasm (`p`).

The different exchange reactions make sure, compounds can move from one compartment to the other. 
The annotation of what receptors a cell has, i.e. what compounds can get from its environment is quite a task but, this is a story for another time! 

### Reaction types

We can distinguish reactions based on the compartments of their metabolites

#### Internal reactions

Reactions that occur only in the cytosol.

In [ ]:
ldh = bt_model.reactions.rxn00499
ldh

#### Transport reactions

Reactions that take extracellular metabolites and transport them into the cell (i.e. convert them into cytosol metabolites)

In [ ]:
gluT = bt_model.reactions.rxn05573
gluT

In [ ]:
print(gluT.build_reaction_string(use_metabolite_names=True))
print(gluT.build_reaction_string(use_metabolite_names=False))

print(gluT.upper_bound)

print(gluT.lower_bound)

#Notice that 'cpd00027_e' is converted to 'cpd00027_c'.

#### Exchange reactions

Reactions that make a metabolite available in the extracellular space. From the extracellular space it can be taken up by a transporter.

In [ ]:
gluExch = bt_model.reactions.EX_cpd00027_e
gluExch

In [ ]:
print(gluExch.build_reaction_string(use_metabolite_names=1))

print(gluExch.lower_bound)

print(gluExch.upper_bound)

# Notice that the inward flux is negative. If the flux is positive, it means the metabolite is getting secreted.

#### Sink reactions

Reactions that eliminate a dead-end metabolite (i.e. a metabolite that is not consumed by any further reaction and is not secreted)

In [ ]:
piSk = bt_model.reactions.piSink

print(piSk.build_reaction_string(use_metabolite_names=1))

print(piSk.lower_bound)

print(piSk.upper_bound)

#### Objective function

In the framework of metabolic models, the objective function is a mathematical expression that represents the biological *goal* of our model, meaning
what our model is trying to do best, i.e. what is it trying to either minimize or maximize.

It is typically defined as a weighted sum of reaction fluxes and is used as the target for optimization in constraint-based modeling approaches like Flux Balance Analysis (FBA), meaning that you can consider this as a list of weights as many as the number of the reactions of your model, highlighting whether and how much a reaction is part of the *goal* of your model. 

To check whether a reaction is part of the objective, we can use an attribute of `cobra`'s `reaction` that was not shown earlier, the 

In [49]:
bt_model.reactions.rxn00545.objective_coefficient

0.0

Since it is 0 this reaction is not part of our objective.
We can use the `objective coefficient` to get which reaction that is about to be optimized by the model.

In [44]:
for i in bt_model.reactions:
    if i.objective_coefficient==1:
        print(i.id)

biomass


> **NOTE**
> 
> It is quite common to add a **pseudo** reaction in our model, called **biomass function**, that represents what compounds and how much of each of them the species requires in order to produce 1 gram of dry weight (gDW) of the organism. 
> If we set this biomass function as a model's objective function, we will be able to check:
> - whether our species grows and what would be its optimal growth rate 
> - what reactions need to perform to achieve this optimal state, and thus what are the limiting factors

In this toy model, we have set a biomass function on our own that looks like this.

In [50]:
bt_model.reactions.biomass

Reaction identifier,biomass
Name,Mock biomass function
Memory address,0x7b9930b90e90
Stoichiometry,3.0 cpd00002_c + 2.0 cpd00004_c + 2.0 cpd00022_c + 2.0 cpd00067_c --> 2.0 cpd00003_c + 3.0 cpd00008_c + 2.0 cpd00010_c 3.0 ATP + 2.0 NADH + 2.0 Acetyl-CoA + 2.0 H+ --> 2.0 NAD + 3.0 ADP + 2.0 CoA
GPR,
Lower bound,0.0
Upper bound,1000.0


What do you think about the bounds ?

### `bt_model` check! 

The keen-eyed read will notice that the number of reactions (33) is not like the one in the `.txt` file (23).

With an easy check, we can also see that the last three reactions from the `.txt` are **not** part of our model, meaning that a total of 13 reactions that are part of our model, they are not in the `.txt` file:
`RNF`, `biomass` and `rxn05654`.

In [37]:
genome_reactions.tail(3)

,ID (BIGG),ID (SEED),name,Reaction,exchange,SEED link,Notes
20,RNF,NaN,Ferredoxin:NAD oxidoreductase ( 2 protons tras...,(3)H+ + NAD + (2) Reduced-ferredoxins <=> NADH...,NaN,http://bigg.ucsd.edu/universal/reactions/RNF,not in found in mSEED
21,biomass,NaN,mock objective,(2) Acetyl-CoA + (3) ATP + (2) NADH + (2) H+ =...,NaN,NaN,NaN
22,NaN,rxn05654,"Transport of succinate, extracellular",Succinate + H+[1] <=> Succinate + H+,Succinate,https://modelseed.org/biochem/reactions/rxn05654,NaN


So, before we move on with how a reaction 
So, as you can see from this summary of your model, it includes a couple of metabolites 
 what's those added reactions ? 

So now, we have a model with several metabolites and reactions, which take place in both the cytosol (`c`) and out of the cell (`e`).



Also, there is an ***objective function***! 

Let's check on this!


In [ ]:
bt_model.reactions.biomass

> **The extra mile**
> 
> This step had a little cheat on our side, since there was no reason to already know which reaction is being used
> as the objective function. 
> And even if we would notice from the name that has something to do with the biomass, we do not know the actual id of the reaction.
> In an actual GEM, to make sure which is the reaction used as the objective function of the model, you would have to check the coefficients of each reaction in the objective function vector
> 
> ```obj_index = [r.objective_coefficient for r in model.reactions].index(True)```
> 

In [8]:
# Getting the reaction id of the objective function in an actual case 
obj_index = [r.objective_coefficient for r in bt_model.reactions].index(True)
bt_model.reactions[obj_index].id

'biomass'

So, what is interesting besides the stoichiometry of the biomass reaction, indicating what compounds are required for biomass to be produced, and also what compounds are being produced, is that there is a **lower** and an **upper bound** in the `Reaction` object.

These values are **essential** for the model as they are those implying its constraints. 

For the biomass function you can see that there is a lower bound of 0 indicating that the model cannot have a negative flux for biomass, i.e. consume biomass, while the upper bound can be considered free, since it gets an extreme high value.

The total list of the **exchange** reactions present in a model, reaction objects for the metabolites that could **potentially be exchanged** (in or out), are available by running:

In [ ]:
bt_model.exchanges

*Exchange* reactions are a type of **boundary reactions**.
There are three different types of pre-defined **boundary reactions**. 
All of them are **unbalanced pseudo reactions**, that means they fulfill a function for modeling by **adding to or removing metabolites** from the model system but are **not based on real biolog**y. These reaction types include:

* **exchange:** reversible reactions that add to or remove an extracellular metabolite from the extracellular compartment 
* **demand:** irreversible reaction that each consumes an intracellular metabolite. 
* **sink** is similar to an exchange but specifically for intracellular metabolites, i.e., a reversible reaction that adds or removes an intracellular metabolite.

Likewise, we can get the **sink** reactions:

In [ ]:
bt_model.sinks

While, we can also get all the boundary reactions together

In [ ]:
bt_model.boundary

> Task! 
>
> Could you find whether other reactions except of the exchange ones are part of the boundary reactions of your model ? 
>
> Is there any reaction type that is missing from your boundary reactions?

### *In silico* environment (medium) in `cobra`

However, only if a metabolite is available in the `medium` of the model can be actually used. 
`medium` contains the information about the concentration of metabolites that are available to the model from the external compartment (media metabolites).


In [ ]:
bt_model.medium

For a human readable version of the same information, one may run:

In [ ]:
for rxn in bt_model.medium:
    cobra_rxn = bt_model.reactions.get_by_id(rxn)
    cobra_met = cobra_rxn.metabolites
    print("reaction string:", cobra_rxn.build_reaction_string(), "metabolites:", [x.name for x in cobra_met])

However, the medium in which your species would grow can be different from the one it brings with it by default.
Yet, the way `cobra` builds the `medium` attribute, you cannot edit it directly.

In [23]:
print(bt_model.medium["EX_cpd00027_e"])
bt_model.medium["EX_cpd00027_e"] = 10 
print(bt_model.medium["EX_cpd00027_e"])

1000.0
1000.0


### Model's solution 

Tomorrow we will discuss in depth several [metabolic modeling analysis methods and some core ideas on what is called *constraint-based analysis*](./computationalMethods.ipynb).

For now, we will think of the most trivial analysis, called Flux Balance Analysis (FBA), as a black box, in order to learn:

- how we can benefit out of it 
- how to get it and handle its findings using `cobra`.

The solution 

In [38]:
solution = bt_model.optimize()
solution

,fluxes,reduced_costs
rxn05573,370.370370,0.000000e+00
rxn00216,370.370370,0.000000e+00
rxn00558,370.370370,0.000000e+00
rxn00545,370.370370,-1.110223e-16
rxn00786,370.370370,0.000000e+00
...,...,...
piSink,-888.888889,-0.000000e+00
h2oSink,1000.000000,5.925926e-01
xSink,-259.259259,-0.000000e+00
nadSink,370.370370,0.000000e+00


Let us know check what would change if our Bt model would not "care" for maximizing its growth, but its ATP production!
ATP production is described in the rxn00148 ModelSEED reaction.

In [18]:
print(bt_model.reactions.rxn00148.build_reaction_string())
for met in bt_model.reactions.rxn00148.metabolites:
    print(met.name)

cpd00002_c + cpd00020_c <=> cpd00008_c + cpd00061_c + cpd00067_c
Pyruvate
ATP
ADP
Phosphoenolpyruvate
H+


The key point here is to observe which compounds are considered as **reactants** and which as **products**!

In [13]:
sol = bt_model.optimize()

In [16]:
sol.fluxes.loc["rxn00148"]

np.float64(-740.7407407407408)

We need to change the objective function to the one for ATP production and then solve the linear program once again.

In [25]:
bt_model.objective = bt_model.reactions.rxn00148

In [26]:
sol_atp = bt_model.optimize()

In [27]:
bt_model.objective_direction = "min"
sol_atp_min = bt_model.optimize()

So, let use see now the optimal solution and the fluxes of the model for maximizing ATP production

In [24]:
sol_atp_min

,fluxes,reduced_costs
rxn05573,400.0,0.000000e+00
rxn00216,400.0,-1.110223e-16
rxn00558,400.0,0.000000e+00
rxn00545,400.0,-2.220446e-16
rxn00786,400.0,0.000000e+00
...,...,...
piSink,-800.0,0.000000e+00
h2oSink,1000.0,-1.600000e+00
xSink,-200.0,0.000000e+00
nadSink,400.0,0.000000e+00


> :question: **Task**
>
> Try to use the medium of *Bt* to *Ri* and optimize for growth? Does it grow? 
> 
> If not, could you suggest limiting factors?

> :question: **Task!**
> 
> How would you try to edit your medium ? 
>
> Try to first edit the bound of an exchange reaction already part of the medium. 
>
> Then, try to remove one and add another. :rocket:
<!-- :hammer: -->

Now, let us have a quick overview of the different ways to analyze a metabolic model, where as first thing we'll investigate how this growth-check we did for the differenet media works! :tada:

## Bonus quest: exploring random media

1. Get the three toy models

In [ ]:
# @title
sugar_fermenter = cobra.io.read_sbml_model("files/models/sugar_fermenter_toy_model.xml")
butyrate_producer = cobra.io.read_sbml_model("files/models/butyrate_producer_toy_model.xml")
acetogen = cobra.io.read_sbml_model("files/models/acetogen_toy_model.xml")

2. Find their exchange reactions

In [ ]:
# @title
def findExchanges(model):
    for reaction in model.reactions:
        if 'EX_' in reaction.id: #alternatively use i in model.exchanges
            reac_string = reaction.build_reaction_string(use_metabolite_names =True)

            print(f"{reaction.id}\t{reac_string}")


print("Sugar fermenter:")
findExchanges(sugar_fermenter)

print("\nButyrate producer:")
findExchanges(butyrate_producer)

print("\nAcetogen:")
findExchanges(acetogen)

3. Make a medium based on their exchange reactions like a python dictionary
`[exchange_reaction] : -10`

`-10` is an arbitrary number representing the maximum amount of the compound that the strain can import from the enviroment.

In [ ]:
# @title
def makeMediumFromModels(modelList):
    medium = {}

    for model in modelList:
        for reaction in model.exchanges:
            medium[reaction.id] = -10

    return medium

medium = makeMediumFromModels([sugar_fermenter, butyrate_producer, acetogen])
print(medium)

4. Make a function to generate a random medium for these exchange reactions

In [ ]:
# @title
import numpy as np

def makeRandomMedium(medium_dict):
    return {reaction: -1*np.random.uniform() for reaction in medium_dict}



med1 = makeRandomMedium(medium)
med2 = makeRandomMedium(medium)
med3 = makeRandomMedium(medium)

print(f"{med1}\n\n{med2}\n\n{med3}")

5. Make a function to apply a medium to a model
   ```python
   def applyMedium(model, medium_dict):
    #return the growth rate of the model in this medium
   ```

In [ ]:
# @title
def applyMedium(model, medium_dict):
    for reaction in medium_dict:
        if model.reactions.has_id(reaction):
            model.reactions.get_by_id(reaction).lower_limit = medium_dict[reaction]

    solution = model.optimize()
    return solution.objective_value

print(f"sugar fermenter: {applyMedium(sugar_fermenter, med1)}")
print(f"\nbutyrate producer: {applyMedium(butyrate_producer, med1)}")
print(f"\nacetogen: {applyMedium(acetogen, med1)}")


6. Apply 1000 random medias to each model and store their growth rates, visualize with an histogram

In [ ]:
# @title
sf = np.zeros(1000)
bp = np.zeros(1000)
ac = np.zeros(1000)

for i in range(1000):
    medium = makeRandomMedium(medium)
    sf[i] = applyMedium(sugar_fermenter, medium)
    bp[i] = applyMedium(butyrate_producer, medium)
    ac[i] = applyMedium(acetogen, medium)

import matplotlib.pyplot as plt

plt.hist(sf, 25, density=True, color = 'red')
plt.show()
plt.hist(bp, 25,  density=True, color = 'blue')
plt.show()
plt.hist(ac, 25,  density=True, color = 'orange')
plt.show()